# LangChain完全入門 Chapter2

Date: 2024/08/15

LangChain完全入門のコードが古いので、私なりにコード部分を更新。

私のアカウントでは以下の２つを利用可能と設定：
- gpt-3.5-turbo
- gpt-3.5-turbo-instruct

In [2]:
# P52
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": "iPhone8のリリース日を教えて"}
    ]
)

for choice in response.choices:
    print(choice.message.content, end='\n\n')

iPhone8は2017年9月22日にリリースされました。



In [10]:
# P52
from openai import OpenAI

client = OpenAI()

try:

    response = client.chat.completions.create(
        model="gpt-3.5-turbo-instruct",
        messages=[
            {"role": "user", "content": "iPhone8のリリース日を教えて"}
        ]
    )
    
    for choice in response.choices:
        print(choice.message.content, end='\n\n')

except Exception as e:
    print(e)

Error code: 404 - {'error': {'message': 'This is not a chat model and thus not supported in the v1/chat/completions endpoint. Did you mean to use v1/completions?', 'type': 'invalid_request_error', 'param': 'model', 'code': None}}


Errorになる原因は以下を参照。instructの方はレガシーのAPIを使っている。

https://stackoverflow.com/questions/75774873/openai-api-error-this-is-a-chat-model-and-not-supported-in-the-v1-completions

## 中間まとめ

Model I/Oモジュールには以下のサブモジュールがある：
- Laguage models
- Prompts
- Output parsers

## Language modelsを使ってみる

In [16]:
# P55
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage

chat = ChatOpenAI(model="gpt-3.5-turbo")

result = chat.invoke([HumanMessage(content="こんにちは！")])
print(result.content)

こんにちは！元気ですか？何かお手伝いできることはありますか？


読んで文字のごとく、HumanMessageは人からLLMへ送られるメッセージ

In [17]:
result = chat.invoke(
    [
        SystemMessage(content="あなたは親しい友人です。返答は敬語を使わず、フランクに会話してください"),
        HumanMessage(content="こんにちは！"),
    ]
)
print(result.content)

こんにちは！元気？


In [18]:
result = chat.invoke(
    [
        SystemMessage(content="あなたは軍人です。部下からの気軽なあいさつへ厳しく答えてください。"),
        HumanMessage(content="こんにちは！"),
    ]
)
print(result.content)

報告！気軽なあいさつは許可されていません。任務に専念せよ！


なるほど、SystemMessageでLLMの役割を設定すると返答内容が変わるのか。

## PromptTemplate

In [21]:
#59

from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate(
    template = "{product}はどこの会社が開発した製品ですか？",
    input_variables = [
        "product"
    ]
)

prompt.format(product="iPhone SE")

'iPhone SEはどこの会社が開発した製品ですか？'

In [24]:
#62

# P52
from openai import OpenAI

client = OpenAI()

humanMessage = HumanMessage(content=prompt.format(product="iPhone SE"))

result = chat.invoke([humanMessage])
print(result.content)

iPhone SEはアメリカの企業であるApple（アップル）が開発した製品です。


勉強してるとき、気分転換にBluesHarp吹くようになった。
以下は自作の3DCG。

<img src="PlayingBluesHarp.jpg" width=400>

今回もBlues Harpを題材に。

In [25]:
humanMessage = HumanMessage(content=prompt.format(product="East top T008k"))

result = chat.invoke([humanMessage])
print(result.content)

East top T008kは、Easttop Musical Instrument Co., Ltd.が開発した製品です。Easttop Musical Instrument Co., Ltd.は、中国のハモンドハーモニカメーカーです。


In [30]:
bluesHarpModels = ["East top 008k", "Lee Oskar harmonica", "Hohner Special 20"]

for harmonicaModel in bluesHarpModels:
    humanMessage = HumanMessage(content=prompt.format(product=harmonicaModel))
    print(chat.invoke([humanMessage]).content, end='\n\n')

East top 008kは、中国の会社であるEasttop Musical Instrument Co., Ltd.が開発した製品です。Easttop Musical Instrument Co., Ltd.は、ハーモニカやその他の楽器を製造している企業です。

Lee Oskar harmonicasは、米国の会社である Tombo Musical Instruments が開発した製品です。Lee Oskarは、アメリカのハーモニカ奏者であり、Tombo Musical Instrumentsと共同で設計されたハーモニカのブランドです。

Hohner Special 20は、ホーナー社が開発したハーモニカの製品です。ホーナー社は、世界中で有名なハーモニカメーカーであり、Hohner Special 20は同社の人気製品の一つです。



毎度、結果が微妙に異なるので、以下にコピーしておく。内容は正確。Lee OskarってトンボのMajor Boyのアメリカ向けなのだと想像出来た。

1回目
```
East top 008kは中国の会社であるEast topが開発した製品です。East topは、電子機器やスマートフォンアクセサリーを製造している企業です。

Lee Oskar harmonicaは、米国の会社である Tombo Musical Instruments が開発した製品です。Lee Oskarは、トンボ社と提携してハーモニカの製品ラインを展開しています。

Hohner Special 20は、ドイツの楽器メーカーであるHohner社が開発したハーモニカです。Hohner社は、世界でも有名なハーモニカメーカーの一つであり、高品質な楽器を製造しています。
```

2回目
```
East top 008kは、中国の会社であるEasttop Musical Instrument Co., Ltd.が開発した製品です。Easttop Musical Instrument Co., Ltd.は、ハーモニカやその他の楽器を製造している企業です。

Lee Oskar harmonicasは、米国の会社である Tombo Musical Instruments が開発した製品です。Lee Oskarは、アメリカのハーモニカ奏者であり、Tombo Musical Instrumentsと共同で設計されたハーモニカのブランドです。

Hohner Special 20は、ホーナー社が開発したハーモニカの製品です。ホーナー社は、世界中で有名なハーモニカメーカーであり、Hohner Special 20は同社の人気製品の一つです。
```

## Output Parser

In [36]:
# P65
from langchain_openai import ChatOpenAI
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain_core.messages import HumanMessage

chat = ChatOpenAI(model="gpt-3.5-turbo")

result = chat.invoke([HumanMessage(content="世界で最も人気のあるブルースハープを8個教えてください")])
print(result.content)

1. Hohner Special 20
2. Hohner Marine Band
3. Lee Oskar Major Diatonic
4. Suzuki Bluesmaster
5. Seydel Session Steel
6. Hohner Blues Harp
7. Fender Blues Deluxe Harmonica
8. Easttop T008K Blues Harmonica


In [38]:
output_parser = CommaSeparatedListOutputParser()

result = chat.invoke([
    HumanMessage(content="世界で最も人気のあるブルースハープを8個教えてください"),
    HumanMessage(content=output_parser.get_format_instructions())
])
output = output_parser.parse(result.content)
for bh in output:
    print(f'代表的な製品 => {bh}')

代表的な製品 => Hohner Special 20
代表的な製品 => Hohner Marine Band
代表的な製品 => Lee Oskar Major Diatonic
代表的な製品 => Seydel Blues Session Steel
代表的な製品 => Suzuki Promaster
代表的な製品 => Lee Oskar Natural Minor
代表的な製品 => Hohner Golden Melody
代表的な製品 => Suzuki Manji


結果は正しい。

## In-memory Cache

In [51]:
from langchain_openai import ChatOpenAI
from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache

chat = ChatOpenAI(model_name="gpt-3.5-turbo", n=2)

set_llm_cache(InMemoryCache())

In [52]:
import time

Q = "Special 20 はどこの会社が開発したハーモニカですか？"

start = time.time()
result = chat.invoke([HumanMessage(content=Q)])
end = time.time()
print(result.content)
print(f"Elapsed time: {end - start:.4f} sec")

Special 20 は、ドイツのホーアー社（Hohner）が開発したハーモニカです。
Elapsed time: 1.4400 sec


In [53]:
start = time.time()
result = chat.invoke([HumanMessage(content=Q)])
end = time.time()
print(result.content)
print(f"Elapsed time: {end - start:.4f} sec")

Special 20 は、ドイツのホーアー社（Hohner）が開発したハーモニカです。
Elapsed time: 0.0014 sec


## Streaming

注意：InMemoryCacheが効いていると2回目はCallBackされないよう。

In [60]:
from langchain_core.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

chat = ChatOpenAI(
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler()
    ]
)

result = chat.invoke(
    [
        HumanMessage(content="とてもおいしいガパオライスの作り方を教えて")
    ]
)

ガパオライスの作り方は以下の通りです。

【材料】
- 鶏ひき肉 200g
- にんにく 2片（みじん切り）
- たまねぎ 1/2個（みじん切り）
- バジル 1/2カップ
- 赤唐辛子 1本（輪切り）
- 油 大さじ2
- 醤油 大さじ2
- おろししょうが 小さじ1
- おろしにんにく 小さじ1
- ご飯 2杯
- 卵 2個
- 塩・こしょう 適量

【作り方】
1. フライパンに油を熱し、にんにくとたまねぎを炒めます。
2. 鶏ひき肉を加えて炒め、色が変わってきたら醤油を加えて味を調えます。
3. 赤唐辛子、おろししょうが、おろしにんにくを加えてさらに炒めます。
4. バジルを加えて炒め、塩・こしょうで味を調えます。
5. 別のフライパンで卵を溶き、ご飯を加えて炒めます。
6. 皿に盛り付け、その上にガパオの具をのせて完成です。

ぜひお試しください。おいしいガパオライスができると思います。

## Templates

In [63]:
# P78

from langchain_openai import ChatOpenAI
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate

examples = [
    {
        "input": "LangChainはChatGPT・Large Language Model(LLM)の実利用をより柔軟に簡易に行うためのツール群です",
        "output": "LangChainは、ChatGPT・Large Language Model(LLM)の実利用をより柔軟に、簡易に行うためのツール群です。"
    }
]

prompt = PromptTemplate(
    input_variables = ["input", "output"],
    template = "入力: {input}\n出力: {output}",
)

few_shot_prompt = FewShotPromptTemplate(
    examples = examples,
    example_prompt = prompt,
    prefix = "以下の句読点の抜けた入力に句読点を追加してください。追加して良い句読点は「、」「。」のみです。他の句読点は追加しないでください。",
    suffix = "入力: {input_string}\n出力:",
    input_variables = ["input_string"]
)

chat = ChatOpenAI()

formatted_prompt = few_shot_prompt.format(
    input_string = "私はさまざまな機能がモジュールとして提供されているLangChainを使ってアプリケーションを開発しています"
)

result = chat.invoke(formatted_prompt)
print("formated_prompt: ", formatted_prompt)
print("result: ", result)

formated_prompt:  以下の句読点の抜けた入力に句読点を追加してください。追加して良い句読点は「、」「。」のみです。他の句読点は追加しないでください。

入力: LangChainはChatGPT・Large Language Model(LLM)の実利用をより柔軟に簡易に行うためのツール群です
出力: LangChainは、ChatGPT・Large Language Model(LLM)の実利用をより柔軟に、簡易に行うためのツール群です。

入力: 私はさまざまな機能がモジュールとして提供されているLangChainを使ってアプリケーションを開発しています
出力:
result:  content='私は、さまざまな機能がモジュールとして提供されているLangChainを使って、アプリケーションを開発しています。' response_metadata={'token_usage': {'completion_tokens': 47, 'prompt_tokens': 217, 'total_tokens': 264}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-66c0617b-4e3f-49aa-a3a1-788578a495bb-0' usage_metadata={'input_tokens': 217, 'output_tokens': 47, 'total_tokens': 264}


## Output Parser

なぜ、f-stringでなくPromptTemplateを使うべきなのか？ChatGPTへ聞いてみた。Prompt Injection 攻撃への対処などが理由。

```
Regarding progmramming with LangChaing, why do i have to use PromptTemplate instead of f-string?

ChatGPT:
ChatGPT
LangChainのプログラミングにおいて、PromptTemplateを使用する理由はいくつかあります。以下は、f-stringを使用する場合との違いを説明します。

1. 再利用性と可読性
PromptTemplateは、複数の異なるプロンプトで再利用可能なテンプレートを定義できます。たとえば、共通のフォーマットを使いたい場合、テンプレートを使うことでコードが整理され、可読性が向上します。
一方、f-stringはシンプルで使いやすいですが、特定のプロンプトに特化しており、再利用性に欠ける場合があります。
2. デバッグと保守
PromptTemplateはテンプレートの形式を検証する機能があり、誤りが発生する可能性を減らします。さらに、テンプレートが複雑な場合、エラーのデバッグが容易になります。
f-stringでは、誤ったフォーマットやキーが見つからない場合にエラーが発生しやすく、デバッグが難しくなることがあります。
3. 外部データの安全な挿入
PromptTemplateは、ユーザー入力や外部データを安全に挿入するためのメカニズムを提供します。これにより、インジェクション攻撃などのセキュリティリスクを軽減します。
f-stringを使用すると、データがそのまま挿入されるため、セキュリティリスクが増加する可能性があります。
4. コンポジションとチェーンのサポート
PromptTemplateは、LangChain内でのプロンプトチェーンや他のコンポーネントと簡単に統合できます。複雑なワークフローの一部として使用するのに適しています。
f-stringはシンプルですが、複雑なチェーンを構成する場合には柔軟性が低くなります。
これらの理由から、LangChainではf-stringの代わりにPromptTemplateを使用することが推奨される場合があります。
```

In [66]:
# P82

from langchain_core.prompts.prompt import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.output_parsers.datetime import DatetimeOutputParser

from langchain_core.messages import HumanMessage

output_parser = DatetimeOutputParser()

chat = ChatOpenAI(model="gpt-3.5-turbo")

prompt= PromptTemplate.from_template("{product}のリリース日を教えて")

result = chat(
    [
    HumanMessage(content=prompt.format(product="iPhone SE")),
    HumanMessage(content=output_parser.get_format_instructions())
    ]
)

output = output_parser.parse(result.content)
output

datetime.datetime(20, 4, 15, 8, 0)

In [67]:
print(output)

0020-04-15 08:00:00


In [69]:
print(type(output))

<class 'datetime.datetime'>


書籍内で、datetimeで出力されるとの補足説明が欲しかった。

In [5]:
# P83
from langchain_openai import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from langchain_core.messages import HumanMessage

# pydantic ライブラリは、Pythonに欠けているフィールドのデフォルト値やバリデーション機能を提供するもの
# 今回、私は初めてこれを使う。
from pydantic import BaseModel, Field, validator

In [9]:
chat = ChatOpenAI(model="gpt-3.5-turbo")

class Smartphone(BaseModel):
    release_date: str = Field(description = "スマートフォンの発売日")
    screen_inches: float = Field(description = "スマートフォンの画面サイズ(インチ)")
    os_installed: str = Field(description = "スマートフォンにインストールされているOS")
    model_name: str = Field(description = "スマートフォンのモデル名")

    @validator("screen_inches")
    def validate_screen_inches(cls, field):
        if field <= 0:
            raise ValueError("Screen inches must be a positive number")
        return field

parser = PydanticOutputParser(pydantic_object=Smartphone)

result = chat.invoke(
    [
    HumanMessage(content="Androidでリリースしたスマートフォンを1個挙げて"),
    HumanMessage(content=parser.get_format_instructions())
    ]
)

parsed_result = parser.parse(result.content)

print(f"モデル名: {parsed_result.model_name}")
print(f"画面サイズ: {parsed_result.screen_inches}")
print(f"OS: {parsed_result.os_installed}")
print(f"スマートフォンの発売日: {parsed_result.release_date}")

モデル名: Samsung Galaxy S22 Ultra
画面サイズ: 6.7
OS: Android 12
スマートフォンの発売日: 2022-03-15


PydanticOutputParserは色々と役に立ちそう。